# Задание 1

Скрипт для асинхронной обкачки урлов

Написать скрипт для обкачки списка урлов с возможностью задавать количество одновременных запросов, используя асинхронное программирование. Клиент можно использовать любой, например, из aiohttp.

In [3]:
import asyncio
import aiohttp

# Функция для выполнения асинхронного запроса к указанному URL
async def fetch_url(session, url):
    try:
        # Открытие асинхронного HTTP-запроса с использованием aiohttp
        async with session.get(url) as response:
            status = response.status
            data = await response.text()
            print(f"URL: {url}, Status: {status}")
            return data
    except Exception as e:
        # Обработка ошибок при выполнении запроса
        print(f"Error fetching {url}: {e}")
        return None

# Функция для обработки списка URL с ограничением на количество одновременных запросов
async def fetch_all_urls(urls, max_concurrent):
    semaphore = asyncio.Semaphore(max_concurrent)  # Семафор для ограничения числа одновременных запросов

    async def fetch_with_semaphore(session, url):
        async with semaphore:
            return await fetch_url(session, url)

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_with_semaphore(session, url) for url in urls]  # Создание задач для каждого URL
        return await asyncio.gather(*tasks)

# Альтернативный запуск для среды Jupyter или других активных событийных циклов
async def main_async():
    # Список URL для обкачки
    urls = [
        "https://example.com",
        "https://httpbin.org/get",
        "https://jsonplaceholder.typicode.com/posts/1",
        "https://jsonplaceholder.typicode.com/posts/2"
    ]
    max_concurrent_requests = 2  # Максимальное количество одновременных запросов

    # Запуск основного асинхронного процесса для обкачки списка URL
    await fetch_all_urls(urls, max_concurrent_requests)

if __name__ == "__main__":
    try:
        # Использование asyncio.run для стандартной среды
        asyncio.run(main_async())
    except RuntimeError:
        # Альтернативный запуск для Jupyter Notebook
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.get_event_loop().run_until_complete(main_async())


URL: https://example.com, Status: 200
URL: https://jsonplaceholder.typicode.com/posts/1, Status: 200
URL: https://httpbin.org/get, Status: 200
URL: https://jsonplaceholder.typicode.com/posts/2, Status: 200


<ipython-input-3-623deb5673e8>:52: RuntimeWarning: coroutine 'main_async' was never awaited
  asyncio.get_event_loop().run_until_complete(main_async())


# Задание 2

Сравнение использования weakref и слотов

Нужно придумать свои типы с несколькими атрибутами:

- класс с обычными атрибутами
- класс со слотами
- класс с атрибутами weakref

Для каждого класса создается большое число экземпляров и замеряется (сравнивается):
- время создания пачки экземпляров
- время чтения/изменения атрибутов

Результаты замеров оформляются скриншотами c описанием и выводом.

In [5]:
import weakref
import time

# Класс с обычными атрибутами
class RegularClass:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

# Класс с использованием слотов
class SlotsClass:
    __slots__ = ('a', 'b', 'c')
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

# Класс с использованием слабых ссылок
class WeakRefClass:
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c

if __name__ == "__main__":
    NUM_INSTANCES = 100000  # Количество экземпляров для теста

    # Замер времени создания экземпляров класса RegularClass
    start_time = time.time()
    regular_instances = [RegularClass(i, i + 1, i + 2) for i in range(NUM_INSTANCES)]
    regular_time = time.time() - start_time

    # Замер времени создания экземпляров класса SlotsClass
    start_time = time.time()
    slots_instances = [SlotsClass(i, i + 1, i + 2) for i in range(NUM_INSTANCES)]
    slots_time = time.time() - start_time

    # Замер времени создания экземпляров класса WeakRefClass с использованием слабых ссылок
    start_time = time.time()
    weakref_instances = [weakref.ref(WeakRefClass(i, i + 1, i + 2)) for i in range(NUM_INSTANCES)]
    weakref_time = time.time() - start_time

    # Вывод времени создания экземпляров
    print("Время создания экземпляров:")
    print(f"RegularClass: {regular_time:.4f} секунд")
    print(f"SlotsClass: {slots_time:.4f} секунд")
    print(f"WeakRefClass: {weakref_time:.4f} секунд")

    # Замер времени доступа и изменения атрибутов класса RegularClass
    start_time = time.time()
    for instance in regular_instances:
        instance.a += 1
    regular_access_time = time.time() - start_time

    # Замер времени доступа и изменения атрибутов класса SlotsClass
    start_time = time.time()
    for instance in slots_instances:
        instance.a += 1
    slots_access_time = time.time() - start_time

    # Замер времени доступа и изменения атрибутов класса WeakRefClass
    start_time = time.time()
    for weak_instance in weakref_instances:
        instance = weak_instance()
        if instance:
            instance.a += 1
    weakref_access_time = time.time() - start_time

    # Вывод времени доступа к атрибутам
    print("Время доступа к атрибутам:")
    print(f"RegularClass: {regular_access_time:.4f} секунд")
    print(f"SlotsClass: {slots_access_time:.4f} секунд")
    print(f"WeakRefClass: {weakref_access_time:.4f} секунд")


Время создания экземпляров:
RegularClass: 0.1775 секунд
SlotsClass: 0.0589 секунд
WeakRefClass: 0.1840 секунд
Время доступа к атрибутам:
RegularClass: 0.0152 секунд
SlotsClass: 0.0155 секунд
WeakRefClass: 0.0125 секунд
